<a href="https://colab.research.google.com/github/mille-s/GEM24_EvalLLM/blob/main/GEM24_EvalLLM_aiXplain_SM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install Aixplain
from IPython.display import clear_output
! pip install aixplain
clear_output()

In [ ]:
#@title Download and Load human-eval-packaged json, and format contents (triples, text, id)
import json
import codecs
from bs4 import BeautifulSoup
import os


language = "Spanish" #@param["English", "Spanish", "Swahili"]
data = 'regular' #@param["regular", "iaa"]

def format_json(json_path):
  # Open en_regular and parse json
  en_regular_json = json.load(codecs.open(json_path, 'r', 'utf-8'))
  # Print first entry
  # print(json.dumps(en_regular_json[0], indent=4))

  triples_text_pairs = []

  x = 0
  while x < len(en_regular_json):
    # if x < 10:
    # Parse html found in the "input" key
    html = en_regular_json[x]['input']
    soup = BeautifulSoup(html, 'html.parser')
    # Print raw table
    # print(soup.prettify())
    table = soup.find('table')
    # headers = [header.text.strip() for header in table.find_all('th')]
    rows = []
    for row in table.find_all('tr'):
      columns = row.find_all(['td', 'th'])  # Get both <td> and <th>
      row_data = ' '.join([col.text.strip() for col in columns])
      rows.append(row_data)
    triples_formatted = '; '.join(rows[1:]) # exclude header
    # print("Headers:", rows[0])
    # print(rows[1:])
    triples_text_pairs.append({'id':en_regular_json[x]['id'], 'triples': '"""'+triples_formatted+'"""', 'text': en_regular_json[x]['output']})
    # else:
    #   break
    x += 1
  return triples_text_pairs

if language == "English":
  if data == 'regular':
    if not os.path.exists('en_regular.json'):
      ! gdown 1hOp1_zN2IgGeTnmzngOgIdLRXErchkEo
    triples_text_pairs = format_json('en_regular.json')
  elif data == 'iaa':
    if not os.path.exists('en_iaa.json'):
      ! gdown 1Q69osoEzJJBVqYH9t2ArPAaPAHVPNax5
    triples_text_pairs = format_json('en_iaa.json')

elif language == "Spanish":
  if data == 'regular':
    if not os.path.exists('es_regular.json'):
      ! gdown 14hNTAfMqQI2-K81O9GZ4GkoybI2iO-sh
    triples_text_pairs = format_json('es_regular.json')
  elif data == 'iaa':
    if not os.path.exists('es_iaa.json'):
      ! gdown 1HadgIdWzcb7NTJYELKB-MJ-qa510e3M4
    triples_text_pairs = format_json('es_iaa.json')

elif language == "Swahili":
  if data == 'regular':
    if not os.path.exists('sw_regular.json'):
      ! gdown 1Knp231CEdUQ-SDnJ_oknJDm2dogetSFg
    triples_text_pairs = format_json('sw_regular.json')
  elif data == 'iaa':
    if not os.path.exists('sw_iaa.json'):
      ! gdown 16JFZg8DacuhRvIAUJ0iMVqnvh4nrxZTX
    triples_text_pairs = format_json('sw_iaa.json')

In [ ]:
#@title Print sample datapoint
expected = 0
if language == "English":
  if data == 'regular':
    expected = 8280
    print('Expected: 8280')
  elif data == 'iaa':
    expected = 92
    print('Expected: 92')
elif language == "Spanish":
  if data == 'regular':
    expected = 3240
    print('Expected: 3240')
  elif data == 'iaa':
    expected = 72
    print('Expected: 72')
elif language == "Swahili":
  if data == 'regular':
    expected = 2700
    print('Expected: 2700')
  elif data == 'iaa':
    expected = 60
    print('Expected: 60')
print(f'Found: {len(triples_text_pairs)}')

if not len(triples_text_pairs) == expected:
  print('--------------------------------------------------\n!!! ERROR: mismatch expected/found data points !!!\n--------------------------------------------------')

print(triples_text_pairs[0])

# Triples = '"""Marcus_Aurelius HasChild Fadilla; Marcus_Aurelius StudentOf Alexander_of_Cotiaeum; Marcus_Aurelius Spouse Faustina_the_Younger; Marcus_Aurelius PositionHeld Roman_emperor"""'
# Nice_Text = '''Marcus Aurelius was a student of Alexander of Cotiaeum and had a child named Fadilla. He was married to Faustina the Younger. He was a Roman emperor and died in Vindobona.'''

In [ ]:
#@title Run evaluation (needs aiXplain API key in Parameters)
import os
import json
import csv
import pickle

# PARAMETERS aiXplain
#==========================
os.environ["AIXPLAIN_API_KEY"] = 'insertYourKeyHere'
Gemini_1dot5_Flash = '674b73f06eb563a748561d41' # Code double checked
path_out = 'Gemini_results'
#==========================
from aixplain.factories import AgentFactory
# from aixplain.modules.agent import ModelTool
from aixplain.modules.agent.tool.model_tool import ModelTool

if not os.path.exists(path_out):
  os.makedirs(path_out)

def dumpResults(annotations, path_out):
  results_file = open(os.path.join(path_out, 'All_Gemini_results'), 'ab')
  pickle.dump(annotations, results_file)
  results_file.close()

def callGemini_aiXplain(prompt, model):
  agent = AgentFactory.create(
    name="Assessment of text quality",
	  description="Assessment of text quality",
	  instructions="",
    tools=[
      ModelTool(model=model),
    ],
  )
  agent_response = agent.run(prompt)

  return agent_response

def runEval(triples_text_pairs, model):
  # EN regular splits: range(0, 2750), range(2750, 5500), range(5500, 8240)
  x = 0
  # To get all evaluations
  for x in range(0, len(triples_text_pairs)):
  # To test on a few inputs only
  # while x < 2:
    Triples = triples_text_pairs[x]['triples']
    Nice_Text = triples_text_pairs[x]['text']
    id = triples_text_pairs[x]['id']

    #Prompt (Do not change unless discussed with the GEM-HumEval group)
    prompt = '''
In this task, you will evaluate the quality of the Text in relation to the given Triple Set. How well does the Text represent the Triple Set?  You will be given four specific Dimensions to evaluate against:

Dimensions:"""
No-Omissions: ALL the information in the Triple Set is present in the Text.
No-Additions: ONLY information from the Triple Set is present in the Text.
Grammaticality: The Text is free of grammatical and spelling errors.
Fluency: The Text flows well and is easy to read; its parts are connected in a natural way."""

Important note on No-Omissions and No-Additions: some Triple Set/Text pairs contain non-factual information and even fictional names for people, places, dates, etc. Whether there are omissions and/or additions in a Text is NOT related to factual truth, but instead is strictly related to the contents of the input Triple Set.
Important note on Grammaticality and Fluency: for Grammaticality and Fluency you do not need to consider the input Triple Set; only the intrinsic quality of the Text needs to be assessed.

You need to provide the scores ranging from 1 (indicating the lowest score) to 7 (indicating the highest score) for each of the dimensions and a short justification for each score in the following JSON format:  {"No-Omissions": {"Justification": "", "Score": ""}, "No-Additions": {"Justification": "", "Score": ""}, "Grammaticality": {"Justification": "", "Score": ""}, "Fluency": {"Justification": "", "Score": ""} }.

Make sure to read thoroughly the Triple Set and the '''+str(language)+''' Text below, and assess the four Dimensions using the instructions and template above.

Triple Set: ''' + str(Triples) + "\n" + '''Text: '''+ str(Nice_Text) + "\n\n" + '''
'''
    print(f'Evaluating text #{x}...')

    responseGemini = callGemini_aiXplain(prompt, model)
    # Gemini outputs single quotes but json.loads expects double quotes
    out_json = json.loads(str(responseGemini['data']['output']).replace("'", '"'))
    # Gemini adds a node "query" in the json, which I remove so the outputs are the same as OpenAI's
    triples_text_pairs[x]['scores_Gemini'] = out_json['query']

    # Save individual files as backup
    with open(os.path.join('Gemini_results', 'Gemini_results_'+str(id)), 'ab') as f:
      pickle.dump(triples_text_pairs[x], f)

    x += 1

  return triples_text_pairs

annotations = runEval(triples_text_pairs, Gemini_1dot5_Flash)

# dumpResults(annotations, path_out)

In [ ]:
#@title Zip and download GPT_results folder
from google.colab import files
import datetime

date = datetime.date.today().strftime("%y%m%d")
zip_name = f'{date}_{language}_GeminiFlash1dot5_{data}_{str(len(triples_text_pairs))}.zip'
# zip_name = f'{date}_{language}_{model}_{data}_0001-2750.zip'

!zip -r /content/{zip_name} /content/Gemini_results
files.download(zip_name)

## Results analysis

In [ ]:
#@title Unzip uploaded file
!unzip /content/GPT_results.zip -d /content

In [ ]:
#@title Load file
import pickle
import glob
import json

def loadAllData(dbfile):
  # load data with pickle
  datapoints = pickle.load(dbfile)
  for dp in datapoints:
    if 'scores_Gemini' in dp:
      print(dp['id'])

# dbfile_all = open('/content/content/GPT_results/All_GPT_results', 'rb')
# loadAllData(dbfile_all)

print('----------------')

def loadDataPoint(dbfile_x):
  # load data with pickle
  dp = pickle.load(dbfile_x)
  if 'scores_Gemini' in dp:
    print(dp['id'])
    print(dp['triples'])
    print(dp['text'])
    print(dp['scores_Gemini'])
    # pickle.load also uses single quotes, whereas json.load expects double quotes
    scores_json = json.loads(str(dp['scores_Gemini']).replace("'", '"'))
    print(f"Gram: {scores_json['Grammaticality']['Score']}; Flu: {scores_json['Fluency']['Score']}; NoOm: {scores_json['No-Omissions']['Score']}; NoAd: {scores_json['No-Additions']['Score']}.")
    print('')

eval_files = glob.glob('Gemini_results/Gemini_results_*')
for filepath in eval_files:
  print(filepath)
  dbfile_x = open(filepath, 'rb')
  loadDataPoint(dbfile_x)